# Summary of work

This notebook draws preliminary comparisons between the UMAP embeddings, a currently generated t-SNE embedding, and an earlier t-SNE embedding by Joris Louwen. 

Because the interactive plots created with plotly inflated the notebook beyond the data limit of Github, I've created HTML versions of the plots and uploaded them via the Bioinformatics server. You can view them via the links provided below the code for the plots.

# Import libraries, data and functions

In [5]:
import pickle
import gensim
import spec2vec
import numpy as np
import pandas as pd

import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import IncrementalPCA

np.random.seed(42)

import seaborn as sns
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from processing import get_ids_for_unique_inchikeys

In [6]:
def colour_dict(var_column, blackout_list=[]):
    # Creates a colour dictionary with 20 distinct colours. Blackout_list offers the possibility to directly assign the colour black to groups.  
    # If the number of groups in the variable column exceeds 20, the only largest groups will be given the distinct colours.
    # If the number of groups is smaller than 20, the first n colours will be used.
    # Colours by Sasha Trubetskoy at https://sashamaps.net/docs/resources/20-colors/
    
    colours = ['#E6194B', '#3CB44B', '#FFE119', '#4363D8', '#F58231',
                '#911EB4', '#46F0F0', '#F032E6', '#BCF60C', '#FABEBE',
                '#008080', '#E6BEFF', '#9A6324', '#0000CD', '#800000', # replaced FFFAC8 (beige) with #0000CD (medium blue) for better visibility against light background
                '#AAFFC3', '#808000', '#FFD8B1', '#000075', '#808080']
    
    groups = list(var_column.value_counts(ascending=False).index)
    for elem in blackout_list:
        groups.remove(elem)
    while (len(groups) > len(colours)):
           colours.append("#000000")
    colour_dict = dict(zip(groups, colours))
    for elem in blackout_list:
        colour_dict[elem] = "#000000"
    return colour_dict

def collapse_classes(df, var, top_n = 20):
    # Collapses all the minority classes of a variable into one super group, to increase legibility of plots. 
    # It returns a dataframe with the var column altered to reflect the directed change.
    top_classes = list(df[var].value_counts(ascending = False).index[:top_n])
    df.loc[~df[var].isin(top_classes), var] = "Other"
    return df

In [7]:
data_dir = "C:/Users/Artur/Documents/Werk/Spec2Vec/Data/"
model_dir = "C:/Users/Artur/Documents/Werk/Spec2Vec/Model/"
embedding_dir = "C:/Users/Artur/Documents/Werk/Spec2Vec/Embeddings/"

# For now, we are only using the spectra that were obtained in positive ion-mode
pretrained_model = gensim.models.Word2Vec.load(model_dir+"ALL_GNPS_210409_positive_cleaned_spec2vec_embedding_iter_15.model")
spectra = pd.read_pickle(data_dir+"ALL_GNPS_210409_positive_cleaned_peaks_processed_s2v.pickle")

# Load the class predictions for each inchikey and shorten the inchikey to the first 14 characters. In case of duplicates, we keep the first occurence and drop the others.
# We only use the first 14 characters of the inchikey (the so-called planar inchikey) because MS spectra cannot be used to meaningfully distinguish compounds beyond these features.
inchikey_classifications = pd.read_csv(data_dir+"ALL_GNPS_210409_positive_processed_annotated_CF_NPC_classes.txt", sep = "\t")
inchikey_classifications.rename(columns = {"inchi_key": "inchikey"}, inplace = True) 

# Retrieve metadata

In [ ]:
# What metadata do we have?
spectra[0].metadata

In [8]:
# We retrieve the inchikey and source instrument for all spectra, and look up the predicted classes for the inchikey.
spectrum_id = []
inchikeys = []
instruments = []

for spec in spectra:
    #short_inchikey = spec.get("inchikey")[:14]
    inchikeys.append(spec.get("inchikey"))
    instruments.append(spec.get("instrument"))
    spectrum_id.append(spec.get("spectrum_id"))
spectrum_metadata = pd.DataFrame({"ID": spectrum_id, "inchikey":inchikeys, "instrument": instruments})

# We drop all spectral records without inchikey and match the remaining records with npclassifier and classyfire compound class predictions
spectrum_metadata = spectrum_metadata[spectrum_metadata["inchikey"] != ""]
spectrum_metadata = spectrum_metadata.merge(inchikey_classifications, on = "inchikey", how = "left")
spectrum_metadata["planar_inchi"] = [key[:14] for key in spectrum_metadata["inchikey"]] # add shortened planar inchikey to metadata
spectrum_metadata = spectrum_metadata.reset_index()

inchi_spectra = [spectra[i] for i in spectrum_metadata.index]
print(f'{len(inchi_spectra)} out of {len(spectra)} spectra have an Inchikey')

187152 out of 199780 spectra have an Inchikey


In [30]:
# Retrieve SpectrumDocuments and convert to vectors
spectrum_documents = [spec2vec.SpectrumDocument(s, n_decimals=2) for i, s in enumerate(inchi_spectra)]
spectrum_vectors = pd.DataFrame([spec2vec.calc_vector(pretrained_model, document, intensity_weighting_power=0.5) for i, document in enumerate(spectrum_documents)])

In [9]:
# We retrieve the IDs of spectra with unique planar Inchikeys
unique_inchi = get_ids_for_unique_inchikeys(inchi_spectra)
print(f'{len(unique_inchi)} out of {len(inchi_spectra)} annotated spectra have a unique Inchikey')

16358 out of 187152 annotated spectra have a unique Inchikey


In [32]:
# We save the spectrum_vectors so we can don't have to calculate those again later
spectrum_vectors.to_csv(data_dir+"ALL_GNPS_210409_positive_spectrumvectors_weighted_0.5.csv", header = False, index = False, sep="\t")

In [23]:
spectrum_vectors = pd.read_csv(data_dir+"ALL_GNPS_210409_positive_spectrumvectors_weighted_0.5.csv", sep = "\t", header = None)

In [ ]:
# This is what the metadata we've collected looks like for the first three spectra
# The class predictions are split up between two different classifiers, ClassyFire (cf), and NPClassifier (npc)
spectrum_metadata.iloc[0:3,:].T

# Reduce dimensions with UMAP and t-SNE

In [ ]:
# Reduce dimensions with umap
umap_df = pd.DataFrame(umap.UMAP(n_components=2, a=1, b=1, n_neighbors=15).fit_transform(spectrum_vectors), columns = ["x", "y"]); umap_df.index = spectrum_metadata.index
umap_df = pd.concat([umap_df, spectrum_metadata], axis = 1)

In [40]:
# Reduce dimensions with t-SNE
tsne_df = pd.DataFrame(TSNE(n_components=2, random_state=42).fit_transform(spectrum_vectors), columns = ["x", "y"]); tsne_df.index = spectrum_metadata.index
tsne_df = pd.concat([tsne_df, spectrum_metadata], axis = 1)

In [ ]:
# Combination PCA + t-SNE
vector_pca = IncrementalPCA(n_components = 50, batch_size = 7500).fit_transform(spectrum_vectors)
pca_tsne = pd.DataFrame(TSNE(n_components=2, random_state=42).fit_transform(vector_pca), columns = ["x", "y"]); pca_tsne.index = spectrum_metadata.index
pca_tsne = pd.concat([pca_tsne, spectrum_metadata], axis = 1)

In [ ]:
# We save the UMAP and t-SNE dataframes we made, so we don't have to calculate those again later
umap_df.to_csv(data_dir+"Annotated_weighted0.5_GNPS_210409_positive_UMAP_a1b1_neighbours15.csv", index = True, sep="\t")
tsne_df.to_csv(data_dir+"Annotated_weighted0.5_GNPS_210409_positive_t-SNE.csv", index = False, sep="\t")
pca_tsne.to_csv(data_dir+"Annotated_weighted0.5_GNPS_210409_positive_PCA_t-SNE.csv", index = False, sep="\t")

In [12]:
# Read UMAP and t-SNE dataframes
umap_df = pd.read_csv(embedding_dir+"Annotated_weighted0.5_GNPS_210409_positive_UMAP_a1b1_neighbours15.csv", sep="\t", index_col = 0)
tsne_df = pd.read_csv(embedding_dir+"Python395_sklearn0242_weighted_t-SNE_state42.csv", sep=",", index_col = 0)
tsne_df = pd.concat([tsne_df, spectrum_metadata], axis = 1)

pca_tsne = pd.read_csv(embedding_dir+"Annotated_weighted0.5_GNPS_210409_positive_PCA_t-SNE.csv", sep="\t")

tsne_louwen = pd.read_csv(embedding_dir+"ALL_GNPS_210409_positive_cleaned_peaks_processed_s2v_only_annotated_tsne2D.csv", sep=",", index_col = 0, names = ["x", "y"])
tsne_louwen = tsne_louwen.merge(spectrum_metadata, how = "inner", left_index = True, right_on = "ID")

In [ ]:
# Plot UMAP and 21/7 t-SNE
hue_var = "npc_superclass_results"
plot_df = collapse_classes(df = umap_df, var = hue_var, top_n = 20)
colourdict = colour_dict(plot_df[hue_var], blackout_list=["Other"])

small_marker = 2; big_marker = 3.5

fig = make_subplots(rows=2, cols=2, horizontal_spacing = 0.1, vertical_spacing = 0.1, 
                    subplot_titles=(f'UMAP: all {len(plot_df)} spectra with an Inchikey', 
                                    f'21/7 t-SNE Joris: all {len(plot_df)} spectra with an Inchikey', 
                                    f'UMAP: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey',
                                    f'21/7 t-SNE Joris: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey'))

fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.25, legendgroup="all"), row=1, col=1)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x),
    row=1, col=1)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.25, legendgroup="all",  showlegend=False), row=2, col=1)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=1)
    
plot_df = collapse_classes(df = tsne_louwen, var = hue_var, top_n = 20)
colourdict = colour_dict(plot_df[hue_var], blackout_list=["Other"])
    
fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.5, legendgroup="all", showlegend=False), row=1, col=2)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x, showlegend=False),
    row=1, col=2)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.5, legendgroup="all",  showlegend=False), row=2, col=2)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=2)

fig.update_layout(height=900, width=1600, title_text=f"UMAP versus t-SNE with data colored based on {hue_var}", title_x=0.5, titlefont=dict(size=24), template='ggplot2', legend={'itemsizing': 'constant'})
fig.show()

fig.write_html(data_dir+"UMAP_vs_21_7_tSNE_Joris.html")

View plot at:
https://www.bioinformatics.nl/~vanb001/UMAP_vs_21_7_tSNE_Joris.html

In [ ]:
# Plot UMAP and 21/7 t-SNE
hue_var = "npc_superclass_results"
plot_df = collapse_classes(df = tsne_df, var = hue_var, top_n = 20)
colourdict = colour_dict(plot_df[hue_var], blackout_list=["Other"])

small_marker = 2; big_marker = 3.5

fig = make_subplots(rows=2, cols=2, vertical_spacing = 0.1, horizontal_spacing = 0.1,
                    subplot_titles=(f'3/8 Artur: all {len(plot_df)} spectra with an Inchikey', 
                                    f'21/7 Joris: all {len(plot_df)} spectra with an Inchikey', 
                                    f'3/8 Artur: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey',
                                    f'21/7 Joris: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey'))

fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.1, legendgroup="all"), row=1, col=1)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x),
    row=1, col=1)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.5, legendgroup="all",  showlegend=False), row=2, col=1)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=1)
    
plot_df = collapse_classes(df = tsne_louwen, var = hue_var, top_n = 20)
colourdict = colour_dict(plot_df[hue_var], blackout_list=["Other"])
    
fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.5, legendgroup="all", showlegend=False), row=1, col=2)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x, showlegend=False),
    row=1, col=2)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.5, legendgroup="all",  showlegend=False), row=2, col=2)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=2)

fig.update_layout(height=900, width=1600, title_text=f"21/7 t-SNE Joris versus 3/8 t-SNE Artur with data colored based on {hue_var}", title_x=0.5, titlefont=dict(size=24), template='ggplot2', legend={'itemsizing': 'constant'})
fig.update_yaxes(range=[-45, 45]); fig.update_xaxes(range=[-50, 50])
fig.show()

fig.write_html(data_dir+"tSNE_Artur_vs_21_7_tSNE_Joris.html")

View plot at:
https://www.bioinformatics.nl/~vanb001/tSNE_Artur_vs_21_7_tSNE_Joris.html

In [ ]:
#Plot t-SNE
hue_var = "npc_superclass_results"
plot_df = collapse_classes(df = tsne_df, var = hue_var, top_n = 20)
colourdict = colour_dict(plot_df[hue_var], blackout_list=["Other"])

small_marker = 2; big_marker = 3.5

fig = make_subplots(rows=2, cols=2, vertical_spacing = 0.1, horizontal_spacing = 0.1,
                    subplot_titles=(f'Only t-SNE: all {len(plot_df)} spectra with an Inchikey', 
                                    f'PCA + t-SNE: all {len(plot_df)} spectra with an Inchikey', 
                                    f'Only t-SNE: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey',
                                    f'PCA + t-SNE: all {len(plot_df.iloc[unique_inchi, :])} spectra with a unique planar Inchikey'))

fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.1, legendgroup="all"), row=1, col=1)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x),
    row=1, col=1)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.5, legendgroup="all",  showlegend=False), row=2, col=1)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=1)

    
plot_df = collapse_classes(df = pca_tsne, var = hue_var, top_n = 20)
    
fig.add_trace(go.Scattergl(x=plot_df["x"], y=plot_df["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = small_marker, marker_line=dict(color='grey', width=0.01), opacity = 0.1, legendgroup="all", showlegend=False), row=1, col=2)
for x, group in plot_df.groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = x, mode='markers', marker_color = colourdict[x], marker_size = small_marker, legendgroup=x, showlegend=False),
    row=1, col=2)

fig.add_trace(go.Scattergl(x=plot_df.iloc[unique_inchi]["x"], y=plot_df.iloc[unique_inchi]["y"], name = "All spectra", mode='markers', marker_color = "white", marker_size = big_marker, opacity = 0.5, legendgroup="all",  showlegend=False), row=2, col=2)
for x, group in plot_df.iloc[unique_inchi,:].groupby(hue_var):
    fig.add_trace(go.Scattergl(x=group.x, y=group.y, name = "", mode='markers',  marker_color = colourdict[x], marker_size = big_marker, legendgroup=x, showlegend=False),
    row=2, col=2)

fig.update_layout(height=900, width=1800, title_text=f"t-SNE of spec2vec vectors colored based on {hue_var}", title_x=0.5, titlefont=dict(size=24), template='ggplot2', legend={'itemsizing': 'constant'})
fig.update_yaxes(range=[-45, 45]); fig.update_xaxes(range=[-50, 50])
fig.show()

fig.write_html(data_dir+"tSNE_Artur_vs_PCA_tSNE.html")

View plot at:
https://www.bioinformatics.nl/~vanb001/tSNE_Artur_vs_PCA_tSNE.html